# Fastpages
> Una plataforma de blogs fácil de usar con funciones adicionales para Jupyter Notebooks.

- toc: true 
- badges: true
- comments: true
- sticky_rank: 1
- author: Jeremy Howard & Hamel Husain
- image: images/fastpages.png
- categories: [fastpages, jupyter]

# Introducción

![](https://github.com/fastai/fastpages/raw/master/images/diagram.png "https://github.com/fastai/fastpages")


[Fastpages](https://github.com/fastai/fastpages) es una plataforma que te permite crear y alojar un blog de forma gratuita, sin anuncios y con muchas funciones útiles, como:

* Cree publicaciones que contengan código, salidas de código (que pueden ser interactivas), texto formateado, etc. directamente desde [Jupyter Notebooks](https://jupyter.org/). Las publicaciones en notebooks admiten funciones como:
    * Las visualizaciones interactivas realizadas con Altair siguen siendo interactivas.
    * Ocultar o mostrar la entrada y salida de la celda.
    * Celdas de código contraíbles que están abiertas o cerradas de forma predeterminada.
    * Defina el título, el resumen y otros metadatos a través de celdas de rebajas especiales
    * Posibilidad de agregar enlaces a Colab y GitHub automáticamente.
* Cree publicaciones, incluido el formato y las imágenes, directamente desde documentos de Microsoft Word.
* Cree y edite publicaciones de [Markdown](https://guides.github.com/features/mastering-markdown/) completamente en línea usando el editor de Markdown incorporado de GitHub.
* Inserta tarjetas de Twitter y videos de YouTube.
* Categorización de publicaciones de blog por etiquetas proporcionadas por el usuario para mayor visibilidad.


En esta sección se enseñará los pasos básicos para poder crear su propio blog con fastpages.

# Primeros pasos

## Configurando Fastpages

¡El proceso de configuración de páginas rápidas también está automatizado con [GitHub Actions](https://github.com/features/actions)! Al crear un repositorio a partir de la plantilla de páginas rápidas, se abrirá automáticamente una solicitud de extracción (después de ~ 30 segundos) configurando su blog para que pueda comenzar a funcionar. La solicitud de extracción automatizada lo recibirá con instrucciones como esta:
![Imgur](https://i.imgur.com/JhkIip8.png)

¡Todo lo que tienes que hacer es seguir estas instrucciones (en el PR que recibes) y tu nuevo sitio de blogs estará en funcionamiento!

## Jupyter Notebooks y Fastpages
En esta parte, se muestran características especiales que fastpages proporciona para los notebooks. También puede escribir las publicaciones de su blog con documentos de Word o makdown.

### Opciones a través de FrontMatter¶
La primera celda de su Jupyter Notebook o markdown contiene información preliminar. El tema principal son los metadatos que pueden activar/desactivar opciones en su Notebook. Tiene el formato siguiente:

```
# Title
> Awesome summary

- toc: true
- branch: master
- badges: true
- comments: true
- author: Hamel Husain & Jeremy Howard
- categories: [fastpages, jupyter]
```

**Todas las configuraciones anteriores están habilitadas en esta publicación, ¡para que pueda ver cómo se ven!**

- El campo de resumen (precedido por `>`) se mostrará debajo de su título y también lo utilizarán las redes sociales para mostrar la descripción de su página.
- `toc`: establecer esto en` true` generará automáticamente una tabla de contenido
- `badges`: establecer esto en` true` mostrará los enlaces de Google Colab y GitHub en la publicación de su blog.
- `comments`: establecer esto en` true` habilitará los comentarios. Consulte [estas instrucciones](https://github.com/fastai/fastpages#enabling-comments) para obtener más detalles.
- `autor` esto mostrará los nombres de los autores.
- `categories` permitirá que su publicación sea categorizada en una página de "Etiquetas", donde los lectores pueden navegar por su publicación por categorías.


_Markdown Front Matters tiene un formato similar al de los notebooks. Las diferencias entre los dos se pueden [ver en las páginas rápidas README](https://github.com/fastai/fastpages#front-matter-related-options) ._

### Code Folding

coloque una marca `# collapse-hide` en la parte superior de cualquier celda si desea **ocultar** esa celda de forma predeterminada, pero déle al lector la opción de mostrarla:

In [ ]:
#hide
!pip install pandas altair

In [ ]:
#collapse-hide
import pandas as pd
import altair as alt

coloque una marca `# collapse-show` en la parte superior de cualquier celda si desea **mostrar** esa celda de forma predeterminada, pero déle al lector la opción de ocultarla:

In [ ]:
#collapse-show
cars = 'https://vega.github.io/vega-datasets/data/cars.json'
movies = 'https://vega.github.io/vega-datasets/data/movies.json'
sp500 = 'https://vega.github.io/vega-datasets/data/sp500.csv'
stocks = 'https://vega.github.io/vega-datasets/data/stocks.csv'
flights = 'https://vega.github.io/vega-datasets/data/flights-5k.json'

Si desea ocultar las celdas por completo (no solo contraerlas), [lea estas instrucciones](https://github.com/fastai/fastpages#hide-inputoutput-cells).

In [ ]:
# hide
df = pd.read_json(movies) # load movies data
df.columns = [x.replace(' ', '_') for x in df.columns.values]
genres = df['Major_Genre'].unique() # get unique field values
genres = list(filter(lambda d: d is not None, genres)) # filter out None values
genres.sort() # sort alphabetically

### Gráficos interactivos con Altair

¡Las visualizaciones interactivas realizadas con [Altair](https://altair-viz.github.io/) siguen siendo interactivas!

Dejamos esta celda de abajo sin ocultar para que pueda disfrutar de una vista previa del resaltado de sintaxis en páginas rápidas, que utiliza el [tema de Drácula](https://draculatheme.com/).

In [ ]:
# select a point for which to provide details-on-demand
label = alt.selection_single(
    encodings=['x'], # limit selection to x-axis value
    on='mouseover',  # select on mouseover events
    nearest=True,    # select data point nearest the cursor
    empty='none'     # empty selection includes no data points
)

# define our base line chart of stock prices
base = alt.Chart().mark_line().encode(
    alt.X('date:T'),
    alt.Y('price:Q', scale=alt.Scale(type='log')),
    alt.Color('symbol:N')
)

alt.layer(
    base, # base line chart
    
    # add a rule mark to serve as a guide line
    alt.Chart().mark_rule(color='#aaa').encode(
        x='date:T'
    ).transform_filter(label),
    
    # add circle marks for selected time points, hide unselected points
    base.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_selection(label),

    # add white stroked text to provide a legible background for labels
    base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
        text='price:Q'
    ).transform_filter(label),

    # add text labels for stock prices
    base.mark_text(align='left', dx=5, dy=-5).encode(
        text='price:Q'
    ).transform_filter(label),
    
    data=stocks
).properties(
    width=500,
    height=400
)